# ===== CREAZIONE DATASET FACCE A PARTIRE DAI VIDEO =====#

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r /content/drive/MyDrive/ESM/hidden_tech.zip /content
!unzip hidden_tech.zip
!rm /content/hidden_tech.zip

In [ ]:
!cp -r /content/drive/MyDrive/ESM/Dataset.zip /content
!unzip Dataset.zip
!rm /content/Dataset.zip

In [4]:
# Avvio dello script che costruisce il dataset delle Facce a partire dal dataset dei video
!python /content/hidden_tech/build_dataset.py

LOG: Creando il dataset in ./Faces
Traceback (most recent call last):
  File "/content/hidden_tech/build_dataset.py", line 171, in <module>
    extract_faces_from_dataset_subfolder('fake', 10,  15,  -1, last_processed_video_index)
  File "/content/hidden_tech/build_dataset.py", line 82, in extract_faces_from_dataset_subfolder
    video_names = [f for f in os.listdir(path.join(old_dataset_path, old_dataset_sub_dir)) if path.isfile(path.join(old_dataset_path, old_dataset_sub_dir, f))]
FileNotFoundError: [Errno 2] No such file or directory: './Dataset/fake'


In [5]:
!mkdir Faces_train
!mkdir Faces_test
!mkdir Faces_validation

!mkdir Faces_train/fake
!mkdir Faces_train/real
!mkdir Faces_test/fake
!mkdir Faces_test/real
!mkdir Faces_validation/fake
!mkdir Faces_validation/real

In [6]:
# Script per la suddivisione delle Faces in train, validation e test
import os
import shutil

TRAIN_PERCENTAGE = 70
TEST_PERCENTAGE = 15
VALIDATION_PERCENTAGE = 100 - TRAIN_PERCENTAGE - TEST_PERCENTAGE

faces_fake = [img for img in os.listdir("/content/Faces/fake") if os.path.isfile(f"/content/Faces/fake/{img}")]
faces_real = [img for img in os.listdir("/content/Faces/real") if os.path.isfile(f"/content/Faces/real/{img}")]
faces_real = sorted(faces_real)
faces_fake = sorted(faces_fake)

train_faces_fake = faces_fake[:int(TRAIN_PERCENTAGE/100*len(faces_fake)/10)*10]
train_faces_real = faces_real[:int(TRAIN_PERCENTAGE/100*len(faces_real)/100)*100]

test_faces_fake = faces_fake[len(train_faces_fake):len(train_faces_fake) + int(TEST_PERCENTAGE/100*len(faces_fake)/10)*10]
test_faces_real = faces_real[len(train_faces_real):len(train_faces_real) + int(TEST_PERCENTAGE/100*len(faces_real)/100)*100]

validation_faces_fake = faces_fake[len(train_faces_fake) + len(test_faces_fake):]
validation_faces_real = faces_real[len(train_faces_real) + len(test_faces_real):]

for i in train_faces_fake:
  shutil.move(f"Faces/fake/{i}","Faces_train/fake")

for i in train_faces_real:
  shutil.move(f"Faces/real/{i}","Faces_train/real")

for i in validation_faces_fake:
  shutil.move(f"Faces/fake/{i}","Faces_validation/fake")

for i in validation_faces_real:
  shutil.move(f"Faces/real/{i}","Faces_validation/real")

for i in test_faces_fake:
  shutil.move(f"Faces/fake/{i}","Faces_test/fake")

for i in test_faces_real:
  shutil.move(f"Faces/real/{i}","Faces_test/real")

FileNotFoundError: ignored

# ===== IMPORTAZIONE DEL DATASET DI FACCE GIA' SPLITTATE ===== #

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp /content/drive/MyDrive/ESM/Faces_splitted.tar.gz /content
!tar -xvzf Faces_splitted.tar.gz
!rm /content/Faces_splitted.tar.gz

In [ ]:
!cp -r /content/drive/MyDrive/ESM/hidden_tech.zip /content
!unzip hidden_tech.zip
!rm /content/hidden_tech.zip

# ===== CREAZIONE ED ALLENAMENTO DNN ===== #





In [ ]:
from hidden_tech import graph_utils as graph
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
import tensorflow.keras.layers as layers
import os
import sys
import json
from datetime import datetime

model = 0

prefix = datetime.now().strftime("%m_%d_%Y_%H_%M_%S")

batch_size    = 64 #questo è il limite !
epochs        = 1
learning_rate = 0.0001
locked_layers = 15

log_interval = 100
log_file     = prefix + "_Training.json"

assert os.path.isfile(log_file) == False


list_fake = os.listdir("Faces_train/fake") 
list_real = os.listdir("Faces_train/real")
train_total = len(list_fake) + len(list_real)

list_fake = os.listdir("Faces_validation/fake") 
list_real = os.listdir("Faces_validation/real")
validation_total = len(list_fake) + len(list_real)

img_width = img_height = 224

train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.2, horizontal_flip=True)
val_datagen   = ImageDataGenerator(rescale=1./255)
test_datagen  = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    "Faces_train", 
    target_size = (img_width, img_height), 
    batch_size = batch_size, 
    class_mode = 'categorical')

val_generator = val_datagen.flow_from_directory(
    "Faces_validation",
    target_size = (img_width,img_height), 
    batch_size = batch_size, 
    class_mode = 'categorical')

test_generator = test_datagen.flow_from_directory(
    "Faces_test", 
    target_size = (img_width,img_height), 
    batch_size = batch_size, 
    class_mode = 'categorical', 
    shuffle = False)

# CARICAMENTO CONDIZIONALE DEL MODELLO

print("Inserisci il path completo del modello che vuoi caricare, altrimenti premi invio crearne uno nuovo")
model_path = input()

if( model_path ):
  model = keras.models.load_model(model_path)

  print("Vuoi continuare ad allenare il modello appena caricato? (y/n)")
  choise = input()

  if(choise == 'y' or choise == 'Y'):

    model.fit(
        train_generator,
        batch_size=batch_size,
        epochs=epochs,
        verbose=1,
        callbacks=[
            graph.SaveCallback(log_file, log_interval, epochs, batch_size, learning_rate, log_interval),
            tf.keras.callbacks.ModelCheckpoint(
              filepath= prefix + "_model",
              monitor='val_loss',
              mode='min',
              save_best_only = True
              )
            ],
        validation_split=0.0,
        validation_data=val_generator,
        shuffle=False,
        class_weight=None,
        sample_weight=None,
        steps_per_epoch= train_total // batch_size,
        validation_steps= validation_total // batch_size,
        validation_batch_size=batch_size,
        validation_freq=1,
        max_queue_size=10,
        workers=1,
        use_multiprocessing=False,
    )

else:
  print("Crea da zero il modello")
  base_model = ResNet50(
      weights = 'imagenet', 
      include_top = False, 
      input_shape = (img_width, img_height, 3))

  #blocco i livelli più in basso
  for layer in base_model.layers[:locked_layers]:
    layer.trainable = False

  #aggiungo i livelli
  model = keras.models.Sequential()
  model.add(base_model)
  model.add(layers.GlobalAveragePooling2D())
  
  # discussione sull'aggiunta di un altro strato denso
  model.add(layers.Dense(3000))
  model.add(layers.LeakyReLU(alpha=0.05))
  model.add(layers.Dense(2,activation="softmax"))
  
  model.compile(
      loss = keras.losses.categorical_crossentropy,
      optimizer = tf.keras.optimizers.Adam(
          learning_rate = learning_rate
          ),
      metrics= ["accuracy"]
      )

  model.fit(
          train_generator,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          callbacks=[
              graph.SaveCallback(log_file, log_interval,epochs, batch_size, learning_rate, log_interval),
              tf.keras.callbacks.ModelCheckpoint(
                filepath= prefix + "_model",
                monitor='val_loss',
                mode='min',
                save_best_only = True+
                )
              ],
          validation_split=0.0,
          validation_data=val_generator,
          shuffle=False,
          class_weight=None,
          sample_weight=None,
          steps_per_epoch= train_total // batch_size,
          validation_steps= validation_total // batch_size,
          validation_batch_size=batch_size,
          validation_freq=1,
          max_queue_size=10,
          workers=1,
          use_multiprocessing=False,
      )


#graph.graph_from_training_data(log_file, True, prefix + '_')

In [ ]:
model.evaluate(test_generator, batch_size = batch_size)

# ===== ATTACCO FGSM ===== #


In [ ]:
!pip install foolbox

In [ ]:
from foolbox.models import TensorFlowModel
from foolbox.attacks import FGSM
import numpy as np

In [ ]:
x_true, y_true = next(train_generator)

y_pred = model.predict(x_true)
y_pred = np.argmax(y_pred, -1) # One hot -> Sparse representation (categorical)
y_true = np.argmax(y_true, -1)

accuracy = np.mean(y_true == y_pred)

In [ ]:
from tensorflow import convert_to_tensor

attack = FGSM()

model_foolbox = TensorFlowModel(model, bounds=(0,1))

preclip, x_advs, res = attack(
    model_foolbox, 
    convert_to_tensor(x_true), 
    convert_to_tensor(y_true), 
    epsilons=0.01
    )

x_advs = x_advs.numpy()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(1)
plt.title("Prima immagine del batch prima dell'attacco")
plt.imshow(x_true[0])

plt.figure(2)
plt.title("Prima immagine del batch dopo l'attacco")
plt.imshow(x_advs[0])

plt.figure(3)
plt.title("Immagina della perturbazione (intensificata)")
plt.imshow((x_advs[0] - x_true[0])  *100)
#mse = np.mean((x_true[0] - x_advs[0])**2)
#print('mse =', mse)
plt.show()


In [ ]:
y_pred_advs = model.predict(x_advs)

y_pred_advs = np.argmax(y_pred_advs, -1)

accuracy_2 = np.mean(y_true == y_pred_advs) # Accuratezza dopo aver moddato il set di input
mse = np.mean((x_true - x_advs)**2)    # Quanto abbiamo moddato il set?
# psnr = 10* np.log10((x.max**2)/mse)

print(f'Accuratezza sul batch prima dell\'attacco: {accuracy}\nAccuratezza sul batch dopo l\'attacco: {accuracy_2}')
print(f'MSE tra le immagini del batch prima e dopo l\'attacco: {mse}')